In [92]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains # 模拟鼠标行为用的，不过这里没用到
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import os,re

os.system('cp ./chromedriver /usr/local/bin')
def scroll_chrome(driver):
    for i in range(500,5000,500):
        driver.execute_script("scrollTo(0,"+str(i)+")")
        time.sleep(0.5)

In [6]:
tool_url = 'https://www.tunemymusic.com/zh-cn/'
browser = webdriver.Chrome() # 设置浏览器驱动为chrome
browser.get(tool_url) # 模拟打开网页
browser.maximize_window()
# 如果出现 报错
# WebDriverException: Message: Can not connect to the Service chromedriver
# 那就是 localhost配置的问题，我之前开了 switchhost，然后里面禁用了localhost
# 参考： https://blog.csdn.net/shenshenruoxi/article/details/105659101
# https://blog.csdn.net/chenlei_525/article/details/70209909
# ummmm 然后好了之后，又出现
# This version of ChromeDriver only supports Chrome version 79
# 猜测应该是 浏览器升级了，驱动没升级
# http://chromedriver.storage.googleapis.com/index.html 驱动下载地址

In [9]:
browser.find_element_by_id('startButton').click() # 找到并点击开始按钮

In [12]:
# browser.find_element_by_class_name('MusicServiceBlock FreeText OnlySource').click()

In [13]:
browser.find_element_by_xpath('/html/body/div[6]/div[3]/div/div[2]/div[3]/div[17]').click() # 选择文本输入歌曲列表

In [15]:
# browser.find_element_by_id('FreeTextInputSection').send_keys("abc")

In [16]:
browser.find_element_by_id('songText').send_keys('abcd') # 这种定位方式可以输入

In [19]:
with open('./mySongList.txt', 'r') as f:
    song_list  = f.read()
song_list

'Youth (Acoustic) - Troye Sivan\nHall of Fame - The Script & will.i.am\n年少有为 - 李荣浩\nFun - Troye Sivan\n北京东路的日子 - 华语群星'

In [21]:
browser.find_element_by_id('songText').send_keys(song_list) # 模拟输入 歌曲 

In [22]:
browser.find_element_by_id('FreeTextConfirmInput').click() # 点击 转换歌曲列表

In [23]:
browser.find_element_by_id('Step2Next').click() # 点击 下一步

In [31]:
browser.find_element_by_xpath('/html/body/div[6]/div[3]/div/div[4]/div[3]/div[2]').click() # 找到 apple music并点击，点击后会弹出新窗口

In [33]:
browser.switch_to_window(browser.window_handles[1]) # 将窗口指向 新窗口句柄

In [34]:
browser.find_element_by_id('LoginBtn').click() # 点击登陆 apple id

这里需要登陆 apple id，建议手动输入（自动化也可以，但是id 和 密码 得内置在这个脚本里，不安全）

In [37]:
browser.switch_to_window(browser.window_handles[0]) # 将窗口指回原本的窗口

In [38]:
#等待登陆完后 页面出现 ‘开始移动我的音乐’按钮 其元素可见
# 参考：https://www.cnblogs.com/wanghuijie1/p/11988472.html
# https://www.cnblogs.com/xiao-xue-di/p/11531917.html
ele_id = "Step4Next"
param = (By.ID,ele_id)
WebDriverWait(browser,10).until(EC.visibility_of_element_located(param)) # 等待该元素出现

<selenium.webdriver.remote.webelement.WebElement (session="affba668938a3ca874fe84a3755b34b5", element="96515bbe-75f7-4152-a524-459b04947d7a")>

In [39]:
browser.find_element_by_id(ele_id).click() # 出现后点击

In [74]:
#等待转换完后 页面出现 ‘再次转换’按钮 其元素可见
ele_id = "ConvertAgain"
param = (By.ID,ele_id)
WebDriverWait(browser,10).until(EC.visibility_of_element_located(param)) # 等待该元素出现
print("当前歌单已经转换完成")

当前歌单已经转换完成


In [76]:
def elementExist_id(ele_id):
    '''
    用于判断页面上某个id的元素是否存在/出现
    '''
    try:
        browser.find_element_by_id(ele_id)
        return True
    except:
        return False

In [77]:
def elementExist_class(class_name):
    try:
        browser.find_element_by_class_name(class_name)
        return True
    except:
        return False

In [78]:
# 如果有丢失歌曲，就勾选丢失列表复选框
ele_id = 'MissingCheckBox'
if elementExist_id(ele_id):
    browser.find_element_by_id(ele_id).click()
    print('1')

1


In [81]:
browser.current_url

'https://www.tunemymusic.com/zh-cn/#step5'

In [79]:
# 如果存在丢失的曲目，则将其输入告知用户
ele_class = 'PlaylistItem InnerItem Fail'
elementExist_class(ele_class)

False

不知道为啥，定位不到丢失歌曲的元素，于是直接获取当前页面源码，然后用beautifulSoup来定位

In [91]:
page_html = browser.page_source
soup = BeautifulSoup(page_html, 'html.parser')
soup.find_all('div', class_='PlaylistItem InnerItem Fail')

[<div class="PlaylistItem InnerItem Fail" id="task32"><div class="TransferringSignParent"><div class="TransferringSign"><div class="RollingParent"><div class="lds-rolling" style="width:100%;height:100%"><div></div></div></div></div></div><div class="PlaylistImageParent"><div class="PlaylistImage DefaultImageTrack"></div></div><div class="NameAndDuration"><div class="PlaylistName">年少有为 - 李荣</div><div class="MovedMissing"><div class="Moved">已移动</div><div class="Missing">丢失</div></div></div></div>,
 <div class="PlaylistItem InnerItem Fail" id="task34"><div class="TransferringSignParent"><div class="TransferringSign"><div class="RollingParent"><div class="lds-rolling" style="width:100%;height:100%"><div></div></div></div></div></div><div class="PlaylistImageParent"><div class="PlaylistImage DefaultImageTrack"></div></div><div class="NameAndDuration"><div class="PlaylistName">北京东路的日子 - 华语群星</div><div class="MovedMissing"><div class="Moved">已移动</div><div class="Missing">丢失</div></div></div><

In [110]:
missing_songs = [
    ''.join(re.findall('PlaylistName">(.*?)</div>', str(songTag)))
    for songTag in soup.find_all('div', class_='PlaylistItem InnerItem Fail')
]
missing_songs

['年少有为 - 李荣', '北京东路的日子 - 华语群星']

发现最后一首丢失歌曲的 class和之前的不一样，单独处理一下

In [111]:
''.join(re.findall(
    'PlaylistName">(.*?)</div>',
    str(soup.find_all('div', class_='PlaylistItem InnerItem LastTrackOfPlaylist Fail'))
))

'北京的日子 - 华语星'

In [ ]:
把最后一首丢失歌曲放入丢失列表

In [112]:
missing_songs.append(
    ''.join(re.findall(
    'PlaylistName">(.*?)</div>',
    str(soup.find_all('div', class_='PlaylistItem InnerItem LastTrackOfPlaylist Fail'))
))
)
missing_songs

['年少有为 - 李荣', '北京东路的日子 - 华语群星', '北京的日子 - 华语星']

In [121]:
bar = '-'*25
print('成功转换{}首歌曲，丢失如下{}首:\n{}\n{}'.format( # song_list.split('\n')把字符串转换成list才可以统计歌曲数量
    len(song_list.split('\n')) - len(missing_songs), len(missing_songs),bar,
    '\n'.join(missing_songs)
))

成功转换2首歌曲，丢失如下3首:
-------------------------
年少有为 - 李荣
北京东路的日子 - 华语群星
北京的日子 - 华语星
